# Lösungen zu den Zusatzübungen zum Notebook "Datenanalyse"

☝️ Beachte 1: es gibt beim Programmieren fast immer verschiedene Lösungswege. Deine Lösung mag anders aussehen, aber dennoch zum gewünschten Resultat führen. Das richtige Resultat ist das Wichtigste. 

☝️ Beachte 2:  In diesem Notebook arbeiten wir größtenteils mit einer Datei, die Du in Übung 1 selbst herunterladen musst. Die meisten Lösungen lassen sich folglich erst ausführen, wenn Du den korrekten Pfad zur heruntergeladenen Datei bei `path` eingesetzt hast. 

☝️ Beachte 3:  Anders als im siebten Notebook kommt hier auch die dot-Notation für den Spaltenzugriff (`df.column` statt `df["column"]`) zum Einsatz.

1. Lade Dir den Datensatz mit 1 Million Sätzen aus deutschsprachigen Nachrichtentexten ("News") aus dem Jahr 2022 von der Seite des Projekts [Wortschatz Leipzig](https://wortschatz.uni-leipzig.de/de/download/German) herunter und speichere ihn an einem sinnvollen Ort. Entpacke die Datei (falls das bei Windows auf Anhieb nicht funktioniert, empfiehlt sich das Programm [WinRAR](https://www.winrar.de)) und lies die Datei "deu_news_2022_1M-sentences.txt" mithilfe von pandas ein. Das DataFrame soll `news_df` heißen und aus zwei Spalten (`sentence_id` und `sentence`) sowie 1 Million Zeilen bestehen. Spaltennamen kannst Du mithilfe des `names`-Argument beim Einlesen definieren. Lass Dir die ersten zehn Zeilen ausgeben.
  
   Befinden sich wirklich 1 Million Sätze im DataFrame? Falls dies bei Deinem DataFrame nicht der Fall ist, überleg Dir, was schief gelaufen sein könnte. Denn in der Datei "deu_news_2022_1M-sentences.txt" befinden garantiert 1 Million Sätze.

   💡 Tipp: Zeilen, die ein öffnendes, aber kein schließendes Anführungszeichen beinhalten, führen dazu, dass pandas "\t" am Ende der Zeile literal anstatt als Trennzeichen interpretiert. Dadurch landen mehrere Sätze in *einer* Zeile (nämlich alle bis zum nächsten "schließenden" Anführungszeichen), weswegen schlussendlich weniger als 1 Million Zeilen im DataFrame stehen. Informiere Dich in der Dokumentation von pandas über den Parameter `quoting`, mit dessen Hilfe Du dieses Verhalten übersteuern kannst.

In [1]:
import pandas as pd #Importieren des Moduls

#Spezifizieren des Pfades zur einzulesenden Datei (hier leer, da individueller Speicherort)
path = ""

"""Einlesen der Daten, Spezifizieren der Spaltennamen, sowie Steuern des Verhaltens bei Anführungszeichen:
Der Parameter 'quoting' löst das Problem, dass mehr als ein Satz in einer Zeile landet. Das Problem entsteht bei Sätzen,
die ein öffnendes Anführungszeichen beinhalten, aber kein schließendes. pandas interpretiert dadurch die folgenden Trennzeichen '\t'
literal anstatt als Trennzeichen, bis zum nächsten Anführungszeichen. Indem wir quoting=3 spezifizieren, teilen wir pandas mit,
dass KEINE Anführungszeichen beim Einlesen als literal interpretiert werden sollen, vgl. https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html;
Anstatt einer ominösen 3, kannst Du auch zusätzlich das Modul csv importieren und quoting=csv.QUOTE_NONE spezifizieren,
die beiden Optionen sind gleichbedeutend."""
news_df = pd.read_csv(path, sep="\t", encoding="utf-8", names=["sentence_id", "sentence"], quoting=3)

print(news_df.shape) #Ausgabe der Anzahl an Spalten und Zeilen des DataFrames: das DataFrame sollte 1 Million Zeilen umfassen
news_df.head(10) #Ausgabe der ersten zehn Zeilen

FileNotFoundError: [Errno 2] No such file or directory: ''

2. Finde heraus, ob es in den deutschsprachigen Nachrichten 2022 häufiger um die Ukraine oder Corona ging (unter der Annahme, dass der Datensatz repräsentativ für die deutschsprachigen Nachrichten ist).

In [2]:
ukraine = news_df[news_df.sentence.str.contains("Ukraine")] #Filtern des DataFrames nach dem Vorkommen von "Ukraine" in der Spalte "sentence"
corona = news_df[news_df.sentence.str.contains("Corona")] #Filtern des DataFrames nach dem Vorkommen von "Corona" in der Spalte "sentence"

#Ausgabe der Länge der beiden Sub-DataFrames
print("Sätze mit 'Ukraine':", len(ukraine))
print("Sätze mit 'Corona':", len(corona))

NameError: name 'news_df' is not defined

3. Schaffe eine zusätzliche Spalte im DataFrame, in der die jeweilige Länge des Satzes in Wörtern verzeichnet ist.

   Wie lange ist der längste Satz? Wie lange ist der kürzeste Satz? Und was ist die durchschnittliche Satzlänge?

   Lass Dir den längsten Satz sowie die kürzesten Sätze ausgeben!

In [3]:
news_df["length"] = news_df.sentence.str.count(" ") + 1 #Zählen der Leerschläge und Addieren von 1, um Anzahl Wörter pro Satz auszurechnen
print(news_df.length.max(), news_df.length.min(), news_df.length.mean()) #Ausgabe des maximalen, minimalen und durchschnittlichen Werts in der Spalte "length"

#Filtern des DataFrames danach, dass in der Spale "length" der maximale bzw. minmale Wert verzeichnet steht, Ausgabe der Werte mithilfe der 'values'-Methode (um nur die Werte zu erhalten)
print(news_df[news_df.length == news_df.length.max()].sentence.values)
print(news_df[news_df.length == news_df.length.min()].sentence.values)

NameError: name 'news_df' is not defined

4. Sortiere das DataFrame nach der Länge der Sätze in absteigender Reihenfolge. Welcher Schritt ist anschließend noch sinnvoll?

In [4]:
#Sortieren des DataFrames nach den Werten der Spalte "length" sowie sinnvollerweise Zurücksetzen der Indizes inkl. "droppen" der alten Indizes
news_df = news_df.sort_values("length", ascending=False).reset_index(drop=True)
news_df

NameError: name 'news_df' is not defined

5. Bereinige sämtliche Sätze derart, dass Sonderzeichen von allen Wortanfängen und -enden entfernt werden. Dazu steht Dir die Liste `special_signs` zur Verfügung. Groß- und Kleinschreibung sollst Du beibehalten. Jeder Satz soll auch nach dem Preprocessing als ein string vorliegen. 

    💡 Tipp: Verwende die pandas-eigene, vektorisierte Art der Datenbearbeitung und denk daran, dass Du diese auch in Kombination mit selbst definierten Funktionen verwenden kannst.  
    
    📌 Herausforderung: Verwende maximal eine Zeile zur Bereinigung der Sätze.

In [5]:
special_signs = ['‰', ':', '§', '´', '.', '́', ';', '❓', '‑', '”', ')', ',', '<', '″', '»', '−', '✔', '•', '"', '`', '〉', '†', '*', '>', '&', "'", '‹', '/', '‚', '®', '°', '‒', '▶', '(', '%', '‘', '€', '«', 'Ł', '═', '„', '!', '–', '?', '-', '︎', '—', '“', '·', '…', '‟', '‡','’', '$', 'ł', '~', '™', '›', '+']
specials_signs_str = "".join(special_signs) #Casten in string, da strip-Methode einen string mit zu strippenden Zeichen erwartet

#Definieren einer eigenen Funktion, die unten auf jeden Satz angewandt wird
def preprocessing(sentence):
    words = sentence.split(" ") #Splitten in Wörter
    #Bereinigen der Wörter (leere strings werden durch if-Bedingung übersprungen)
    preprocessed_words = [word.strip(specials_signs_str) for word in words if len(word.strip(specials_signs_str)) > 0] 
    return " ".join(preprocessed_words) #Rückgabe eines wieder als string zusammengesetzten Satzes mit bereinigten Wörtern

#Überschreiben der Spalte "sentence"
news_df["sentence"] = news_df.sentence.apply(preprocessing) #Anwenden der benutzerdefinierten Funktion auf alle Zeilen in der Spalte "sentence"

#Herausforderung
#news_df["sentence"] = news_df.sentence.apply(lambda sentence: " ".join([word.strip(specials_signs_str) for word in sentence.split(" ") if len(word.strip(specials_signs_str)) > 0]))

news_df

NameError: name 'news_df' is not defined

6. Mit welchen zehn Wörtern beginnen die Nachrichtensätze am häufigsten?

   💡 Tipp: Um jeweils auf das n-te Element einer Liste in jeder Zeile einer bestimmten Spalte zuzugreifen, kannst Du `.str[n]` an den Spaltennamen anhängen.

In [6]:
#Splitten der Sätze in Wörter, Indizieren des ersten Elements, Berechnen der Häufigkeitsverteilung und Ausgabe der obersten zehn Elemente
news_df.sentence.str.split(" ").str[0].value_counts().head(10) 

NameError: name 'news_df' is not defined

7. Und mit welchen zehn Wörtern enden die die Nachrichtensätze am häufigsten?

In [7]:
#Splitten der Sätze in Wörter, Indizieren des letzten Elements, Berechnen der Häufigkeitsverteilung und Ausgabe der obersten zehn Elemente
news_df.sentence.str.split(" ").str[-1].value_counts().head(10) 

NameError: name 'news_df' is not defined

8. Wortschatz Leipzig stellt nicht nur eine Million Sätze zur Verfügung, sondern listet auch auf, wann und von welchem Nachrichtenportal die Sätze extrahiert wurden. Diese Informationen sind jedoch in zwei weiteren Dateien gespeichert, die Du ebenfalls bereits heruntergeladen und entpackt hast. Sämtliche Quellen sowie Daten (wann wurde der Satz heruntergeladen?) sind in "deu_news_2022_1M-sources.txt" aufgelistet und mit Quellen-IDs versehen, die Zuordnung zwischen Quellen-ID und Satz-ID findet sich wiederum in "deu_news_2022_1M-inv_so.txt".

    Deine Aufgabe ist es nun, diese Informationen (Quelle und Datum) mit dem DataFrame `news_df` zu vereinen. Dazu bietet sich eine Dir vermutlich noch unbekannte Methode namens `merge` an. Informiere Dich [hier](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) über die Methode. Bei `merge` kannst Du mithilfe des `on`-Parameters spezifizieren, auf Basis der Werte welcher Spalte die Zusammenführung zweier DataFrames erfolgen soll. Wichtig ist dabei, dass der angegebene Spaltenname in beiden DataFrames existiert. Stelle sicher, dass du am Ende immer noch eine Million Sätze in Deinem DataFrame hast, sowie, dass keine fehlenden Werte (die etwa aufgrund eines falschen Mergings eingetragen wurden) darin vorkommen. Verwende dazu entweder `if`-Bedingungen, oder, eleganter, `assert`-Statements (mehr dazu [hier](https://realpython.com/python-assert-statement/#getting-to-know-assertions-in-python)). 
    
    Überprüfe abschließend bei ein paar Quellenlinks, ob die jeweiligen Sätze tatsächlich auf der entsprechenden Website zu finden sind.
    
    ⚠️ Achtung: Wenn Du mehrfach hintereinander dieselben DataFrames zusammenfügst, riskierst Du einen `KeyError` bei der als `on`-Parameter übergebenen Spalte. Die Fehlermeldung rührt daher, dass pandas bei mehrmaligem Mergen bereits existierende Spaltennamen um ein Suffix ergänzt, da Spaltennamen einzigartig sein müssen. Schaffe deshalb als erstes eine Kopie von `news_df` mithilfe von `news_df_enriched = news_df.copy()` und arbeite fortan mit `news_df_enriched`. Jedes Mal wenn Du die ganze Zelle ausführst, um die DataFrames zu mergen, passiert dies somit auf Basis einer (immer wieder) neu erstellten Kopie.

In [8]:
news_df_enriched = news_df.copy()

#Spezifizieren der Pfade zur einzulesenden Datei (hier leer, da individueller Speicherort)
sources = pd.read_csv("", sep="\t", quoting=3, names=["source_id", "source", "date"], encoding="utf-8")
mapping = pd.read_csv("", sep="\t", quoting=3, names=["source_id", "sentence_id"], encoding="utf-8")

news_df_enriched = news_df_enriched.merge(mapping, on="sentence_id") #Zusammenführen von news_df und mapping, basierend auf Werten der Spalte "sentence_id"
news_df_enriched = news_df_enriched.merge(sources, on="source_id") #Zusammenführen von news_df (jetzt mit source_id aus vorangegangenem Merging) und sources, basierend auf Werten der Spalte "source_id"

#Überprüfen, ob Länge von news_df immer noch 1 Million mithilfe des assert-Statements
assert len(news_df_enriched) == 1000000

"""Überprüfen, ob fehlende Werte in news_df vorkommen mithilfe von isna(), das ein DataFrame mit True/False für jede
einzelne Zelle ausgibt. False ist bei Python gleichbedeutung mit 0 (True mit 1), weswegen wir die sum-Methode anhängen können,
die standardmäßig spaltenweise aufsummiert, wobei immer noch jede Spalte Null ergeben sollte. Anschließend können wir auf die
resultierende Series nochmal sum() anwenden, um die Spaltensummen zusammenzuzählen. Ergibt dies immer noch Null, finden sich
keinerlei fehlende Werte in news_df_enriched. Lasse Dir ggf. Zwischenschritte ausgeben, um das Ergebnis nachvollziehen zu können"""
assert news_df_enriched.isna().sum().sum() == 0

#Ausgabe von ein paar Links zur manuellen Überprüfung, ob Sätze tatsächlich aus der jeweiligen Quelle stammen
some_indices = [330, 8374, 99473]
for index in some_indices:
    print(news_df_enriched.loc[index, "sentence"], "sollte von", news_df_enriched.loc[index, "source"], "stammen. Stimmts?")

NameError: name 'news_df' is not defined

9. Füge eine weitere Spalte namens `country` hinzu, die basierend auf der Top-Level-Domain dem Quellenlink (z.B. ".de" oder ".ch") angibt, aus welchem Land der jeweilige Nachrichtensatz stammt. Dies lässt sich am besten mithilfe eines regulären Ausdrucks (vgl. sechstes Notebook) lösen. Pandas wiederum stellt mit `findall` eine nützliche string-Methode zur Verfügung, die wir wie gewohnt auf eine ganze Spalte anwenden können und der wir problemlos einen regulären Ausdruck übergeben können. 

    📌 Herausforderung: Lass Dir eine schön formatierte Übersicht über die absolute und eine relative Häufigkeitsverteilung der Länder ausgeben. Folgender Screenshot ist eine Idee für die Formatierung, Du kannst es aber auch anders umsetzen.
    
    <img src="../../3_Dateien/Grafiken_und_Videos/tld_overview.png" width="300"/>

In [9]:
news_df = news_df_enriched #"Zurückverweisen" von news_df_enriched auf news_df, da dies der simplere Variablenname ist

import re

#Definieren der RegEx: ein Punkt (escaped mit Backslash), gefolgt von zwei oder mehr kleingeschriebenen Buchstaben, gefolgt von Schrägstrich (escaped mit Backslash)
pattern = r"\.[a-z]{2,}\/"

"""Schaffen einer neuen Spalte, die jeweils das erste Element (Index 0) der durch findall(pattern) produzierten Liste 
mit Regex-Matches beinhält (bereinigt vom Punkt zu Beginn und dem Schrägstrich am Ende)"""
news_df["country"] = news_df.source.str.findall(pattern).str[0].str.strip("./") 

#Herausfoderung
#Schaffen zweier dictionaries mit den absoluten und relativen Häufigkeitsverteilungen (dictionaries eignen sich besser als Series für die Iteration unten)
absolute, relative = dict(news_df.country.value_counts()), dict(news_df.country.value_counts(normalize=True))

#Ausgabe einer Überschrift sowie eines horizontalen Trennbalkens, Formatierung mithilfe von f-strings
print(f"{'TLD':10}{'Absolute':>10}{'Relative':>10}\n{'-'*30}")

#Iteration über eines der beiden dictionaries, Ausgabe des Schlüssels sowie der beiden Werte aus den dictionaries, Formatierung mithilfe von f-strings
for key, val in absolute.items():
    print(f"{key:10}{val:10}{relative[key]:10.3f}")

NameError: name 'news_df_enriched' is not defined

10. Schaffe ein Sub-DataFrame mit Nachrichtensätzen aus dem [DACH](https://de.wikipedia.org/wiki/D-A-CH)-Raum.

In [10]:
#Schaffen eines Sub-DataFrames, indem news_df in der Spalte "country" nach den Werten "de", "at" und "ch" gefiltert wird
dach_df = news_df[news_df.country.isin(["de", "at", "ch"])]
dach_df.head() #Ausgabe der obersten Zeilen des Sub-DataFrames

NameError: name 'news_df' is not defined

11. Zum Schluss wollen wir noch überprüfen, ob das Sampling der Nachrichtensätze im Datensatz einigermaßen repräsentativ für die deutschsprachigen Länder ist, wobei wir uns der Einfachheit halber wieder auf den DACH-Raum beschränken. Deutschland ist ungefähr um den Faktor 10 größer als Österreich resp. die Schweiz (überprüfe gerne die aktuellen Einwohner:innenzahl der drei Länder). Entsprechend sollten zirka zehn mal so viele Nachrichtensätze aus Deutschland stammen als aus Österreich bzw. der Schweiz. Gleichzeitig sollte der Anteil an Nachrichtensätzen aus Deutschland, Österreich und der Schweiz aber auch einigermaßen gleichmäßig über das Jahr 2022 verteilt sein. Idealerweise wurden nicht bloß im Januar Schweizer Quellen, im Februar österreichische und den Rest des Jahres deutsche gesammelt... Die Verteilung nach Land über die Monate des Jahres 2022 hinweg wollen wir uns als Plot ausgeben lassen. Bevor Du diesen Plot erstellst, führe folgende Vorbereitungsschritte aus:

    - Wenn Du dir die Daten (wann wurde der Satz heruntergeladen?) in `news_df` anschaust (z.B. mittels `news_df[~news_df.date.str.startswith("2022")])`, siehst Du, dass einige unrealistische Daten dabei sind. Da hat wohl das Web-Scraping bzw. das Postprocessing versagt... Filtere das DataFrame derart, dass nur Sätze aus dem Jahr 2022 übrigbleiben.
    - Da wir die Verteilung anstatt über 365 Tage vereinfacht über zwölf Monate hinweg plotten wollen, schaffe eine neue Spalte mit dem jeweiligen Extraktionsmonat des Nachrichtensatzes. Sortiere das DataFrame anschließend nach den Werten dieser neuen Spalte.
    </br></br>
    
    Analysiere nun folgenden Plot und erstelle ihn anschließend selbst. Überleg Dir abschließend, ob das Sampling repräsentativ für die Größe der DACH-Länder über die Monate hinweg ist.
    
    
    <img src="../../3_Dateien/Grafiken_und_Videos/news_dach.png" width="600"/> </br>

    

In [11]:
import matplotlib.pyplot as plt #Importieren von matplotlib.pyplot

dach_df = dach_df[dach_df.date.str.startswith("2022")] #Wegfiltern von Daten außerhalb des Jahres 2022

dach_df["month"] = dach_df.date.str.split("-").str[1] #Schaffen einer neuen Spalte mit Monaten durch Splitten nach "-" und Indizieren des zweiten Elements

dach_df = dach_df.sort_values("month", ascending=True) #Sortieren des DataFrames nach Monat

tlds = ["de", "at", "ch"] #Definieren einer Liste mit DACH-TLDs

#Iterieren über die TLDs
for tld in tlds:
    tld_df = dach_df[dach_df.country == tld] #Schaffen eines Sub-DataFrames mit Nachrichtensätzen nur aus dem jeweiligen Land
    x = tld_df.month.unique() #Definieren der Werte, die auf der x-Achse geplottet werden (einzigartige Werte in der Spalte "month", d.h. die zwölf Monate)
    """Definieren der Werte, die auf der y-Achse geplottet werden, indem das Sub-DataFrame nach Werten in der Spalte "month" gruppiert wird und für jede Gruppe
    die Anzahl an Zeilen über size() ermittelt wird. Der Wert jeden Monats wird anschließend durch die Gesamtanzahl an Zeilen im jeweiligen Monat in dach_df geteilt,
    um den relativen Anteil des jeweiligen Landes im entsprechenden Monat auszurechnen."""
    y = tld_df.groupby(["month"]).size() / dach_df.groupby(["month"]).size()
    plt.plot(x, y, 'o-') #Plotting im 'o-'-Stil

#Plotten von Titel, Achsenbeschriftungen und Legende
plt.title("Anzahl an Nachrichtensätzen aus den DACH-Ländern")
plt.xlabel("Monate 2022")
plt.ylabel("Anteil an DACH-Quellen/Monat")
plt.legend(tlds, loc="best")

ModuleNotFoundError: No module named 'matplotlib'

Antwort auf abschließende Frage: Das Sampling ist definitiv gleichmäßig über die Monate hinweg. Der jeweilige Anteil der Länder entspricht einigermaßen ihrer Größenordnung, wobei der Anteil der Schweizer Quellen etwas zu niedrig und der Anteil der österreichischen Quellen etwas zu hoch ausfällt.